In [2]:
!pip install pandas nltk pyodbc sqlalchemy

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 8.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pyodbc
import nltk 
from  nltk.sentiment.vader import SentimentIntensityAnalyzer


In [3]:
# Download the VADER lexicon for sentiment analysis if not already present.
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PMLS\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.


In [7]:
#fetch data from sql
sia=SentimentIntensityAnalyzer()
def fetch_data_from_sql():
    # Define the connection string with parameters for the database connection
    conn_str = (
        "Driver={ODBC Driver 17 for SQL Server};"  
        "Server=DESKTOP-8BL3MIG;"  
        "Database=PortfolioProject_MarketingAnalytics;"  
        "Trusted_Connection=yes;"  
    )
    conn=pyodbc.connect(conn_str)

    query='select * from customer_reviews'

    df=pd.read_sql(query,conn)

    conn.close()

    return df

#fetch data from sql database
customer_reviews_df=fetch_data_from_sql()

sia=SentimentIntensityAnalyzer()

#sentiment analysis based on review
def sentiment_score_calculation(review):
    sentiment=sia.polarity_scores(review)
    return sentiment['compound']

def categorize_sentiment(score, rating):
    # Use both the text sentiment score and the numerical rating to determine sentiment category
    if score > 0.05:  # Positive sentiment score
        if rating >= 4:
            return 'Positive'  # High rating and positive sentiment
        elif rating == 3:
            return 'Mixed Positive'  # Neutral rating but positive sentiment
        else:
            return 'Mixed Negative'  # Low rating but positive sentiment
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative'  # Neutral rating but negative sentiment
        else:
            return 'Mixed Positive'  # High rating but negative sentiment
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  # High rating with neutral sentiment
        elif rating <= 2:
            return 'Negative'  # Low rating with neutral sentiment
        else:
            return 'Neutral'  # Neutral rating and neutral sentiment

# Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment

#3 columns
customer_reviews_df['SentimentScore']=customer_reviews_df['ReviewText'].apply(sentiment_score_calculation)

customer_reviews_df['SentimentCategory']=customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

customer_reviews_df.head()


customer_reviews_df.to_csv('customer_reviews_with_sentiment.csv',index=False)










C:\Users\PMLS\AppData\Local\Temp\ipykernel_18440\3005130541.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query,conn)
